In [1]:
!pip install copick git+https://github.com/copick/copick-utils.git scikit-image cupy-cuda12x torch torchvision tqdm matplotlib

  Cloning https://github.com/copick/copick-utils.git to /tmp/pip-req-build-xraid2r_
  Running command git clone --filter=blob:none --quiet https://github.com/copick/copick-utils.git /tmp/pip-req-build-xraid2r_
  Resolved https://github.com/copick/copick-utils.git to commit 79315f3fca71ebfaaff54198b3e08a3bd2ebc63f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 77.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Make a copick project

config_blob = """{
    "name": "czii_cryoet_mlchallenge_2024",
    "description": "2024 CZII CryoET ML Challenge training data.",
    "version": "1.0.0",

    "pickable_objects": [
        {
            "name": "apo-ferritin",
            "is_particle": true,
            "pdb_id": "4V1W",
            "label": 1,
            "color": [  0, 117, 220, 128],
            "radius": 60,
            "map_threshold": 0.0418
        },
        {
            "name": "beta-amylase",
            "is_particle": true,
            "pdb_id": "1FA2",
            "label": 2,
            "color": [153,  63,   0, 128],
            "radius": 65,
            "map_threshold": 0.035
        },
        {
            "name": "beta-galactosidase",
            "is_particle": true,
            "pdb_id": "6X1Q",
            "label": 3,
            "color": [ 76,   0,  92, 128],
            "radius": 90,
            "map_threshold": 0.0578
        },
        {
            "name": "ribosome",
            "is_particle": true,
            "pdb_id": "6EK0",
            "label": 4,
            "color": [  0,  92,  49, 128],
            "radius": 150,
            "map_threshold": 0.0374
        },
        {
            "name": "thyroglobulin",
            "is_particle": true,
            "pdb_id": "6SCJ",
            "label": 5,
            "color": [ 43, 206,  72, 128],
            "radius": 130,
            "map_threshold": 0.0278
        },
        {
            "name": "virus-like-particle",
            "is_particle": true,
            "pdb_id": "6N4V",            
            "label": 6,
            "color": [255, 204, 153, 128],
            "radius": 135,
            "map_threshold": 0.201
        }
    ],

    "overlay_root": "/kaggle/working/test/overlay",

    "overlay_fs_args": {
        "auto_mkdir": true
    },

    "static_root": "/kaggle/input/czii-cryo-et-object-identification/test/static"
}"""

copick_config_path = "/kaggle/working/copick.config"

with open(copick_config_path, "w") as f:
    f.write(config_blob)

In [5]:
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import numpy as np
from skimage.measure import regionprops
from skimage.morphology import ball
from skimage.segmentation import watershed
from tqdm import tqdm
import scipy.ndimage as ndi
import time
import csv
import os
import copick
import zarr

DEVICE = 'cuda'
OUTPUT_CSV_PATH = "submission.csv"
tomo_type = 'denoised'
RESOLUTION_THRESHOLD = 16

def gaussian_kernel(size, sigma):
    """Generate a 3D Gaussian kernel."""
    kernel = np.fromfunction(
        lambda x, y, z: (1/ (2 * np.pi * sigma**2)) * 
        np.exp(- ((x - (size[0] - 1) / 2) ** 2 + 
                   (y - (size[1] - 1) / 2) ** 2 + 
                   (z - (size[2] - 1) / 2) ** 2) / (2 * sigma ** 2)),
        size
    )
    return torch.tensor(kernel).float().unsqueeze(0).unsqueeze(0).to(DEVICE)  # Add batch and channel dimensions

def create_hessian_particle_mask(tomogram, sigma):
    """
    Generate a binary mask for dark, blob-like particles in a cryo-ET tomogram
    using Hessian-based filtering with PyTorch.

    Args:
        tomogram (torch.Tensor): The input 3D tomogram (C, D, H, W).
        sigma (float): The standard deviation for Gaussian smoothing.

    Returns:
        torch.Tensor: Binary mask highlighting dark blob-like areas in the tomogram.
    """
    kernel_size = (5, 5, 5)
    gaussian_k = gaussian_kernel(kernel_size, sigma)
    
    tomogram_smoothed = F.conv3d(tomogram.unsqueeze(0).unsqueeze(0), gaussian_k, padding=2).squeeze()

    # Compute Hessian components
    hessian_xx = F.conv3d(tomogram_smoothed.unsqueeze(0).unsqueeze(0), gaussian_k, padding=2)
    hessian_yy = F.conv3d(tomogram_smoothed.unsqueeze(0).unsqueeze(0), gaussian_k, padding=2)
    hessian_xy = F.conv3d(tomogram_smoothed.unsqueeze(0).unsqueeze(0), gaussian_k, padding=2)

    hessian_response = hessian_xx + hessian_yy + hessian_xy  # Simplified combination
    binary_mask = hessian_response < 0  # Adjust threshold based on your needs

    return binary_mask.squeeze().byte()

def erode_dilate_mask(mask, radius):
    """
    Perform binary erosion and dilation on a binary mask using a spherical structuring element.
    
    Args:
        mask (torch.Tensor): Input binary mask
        radius (int): Radius of the spherical structuring element
        
    Returns:
        torch.Tensor: Dilated mask after erosion and dilation operations
    """
    # Create a spherical structuring element
    radius = int(radius)  # Ensure radius is an integer
    struct_elem = ball(radius)
    struct_elem_tensor = torch.tensor(struct_elem, dtype=torch.float32, device=DEVICE).unsqueeze(0).unsqueeze(0)

    # Reshape mask for conv3d
    mask_reshaped = mask.unsqueeze(0).unsqueeze(0).float()  # Shape (1, 1, D, H, W)
    
    # Calculate padding size - ensure it's an integer
    pad_size = int(radius // 2)
    
    # Debug: Print shapes
    print(f"Mask shape for erosion: {mask_reshaped.shape}")
    print(f"Structuring element shape: {struct_elem_tensor.shape}")
    print(f"Padding size: {pad_size}")
    
    # Erosion: Use a negative structuring element for max pooling
    # Convert padding to the expected format (left, right, top, bottom, front, back)
    # Ensure all values are integers
    pad_3d = (int(pad_size), int(pad_size), 
              int(pad_size), int(pad_size), 
              int(pad_size), int(pad_size))
    
    mask_padded = F.pad(mask_reshaped, pad_3d, mode='constant', value=1)
    eroded = -F.conv3d(
        -mask_padded,
        struct_elem_tensor,
        stride=1,
        padding=0,
        dilation=1,
        groups=1
    )
    eroded = (eroded >= struct_elem_tensor.sum()).squeeze().byte()

    # Dilation
    mask_padded = F.pad(eroded.unsqueeze(0).unsqueeze(0).float(), pad_3d, mode='constant', value=0)
    dilated = F.conv3d(
        mask_padded,
        struct_elem_tensor,
        stride=1,
        padding=0,
        dilation=1,
        groups=1
    )
    dilated = (dilated > 0).squeeze().byte()
    
    return dilated

def distance_transform(mask):
    """
    Compute the distance transform using a simple distance transform approach.
    
    Args:
        mask (torch.Tensor): Binary mask tensor
        
    Returns:
        torch.Tensor: Distance transform result
    """
    # Ensure mask is boolean, then convert to float for distance calculation
    mask = mask.bool()
    # Invert the mask (using logical not instead of bitwise not)
    inverted_mask = (~mask).float()
    
    # Add batch and channel dimensions
    inverted_mask = inverted_mask.unsqueeze(0).unsqueeze(0)
    
    # Create kernel on the correct device
    kernel = torch.ones(1, 1, 3, 3, 3, device=DEVICE)
    
    # Compute distance transform using convolution
    distance = F.conv3d(inverted_mask, kernel, padding=1)
    
    return distance.squeeze()

def local_maxima(distance, radius):
    """
    Detect local maxima in the distance transform.
    
    Args:
        distance (torch.Tensor): Distance transform tensor
        radius (int): Radius for local maxima detection
        
    Returns:
        torch.Tensor: Binary mask of local maxima
    """
    # Ensure radius is an integer
    radius = int(radius)
    
    # Add batch dimension for max_pool3d
    distance = distance.unsqueeze(0)
    
    # Create kernel size tuple (must be odd numbers)
    kernel_size = (2 * radius + 1, 2 * radius + 1, 2 * radius + 1)
    
    # Compute local maxima
    maxpool = F.max_pool3d(
        distance,
        kernel_size=kernel_size,
        stride=1,
        padding=radius
    )
    
    # Compare with original distance to find local maxima
    local_max = (distance == maxpool)
    
    return local_max.squeeze()

def get_tomogram_data(run, voxel_spacing, radius):
    """
    Get tomogram data at appropriate resolution based on particle radius.
    
    Args:
        run: Run object
        voxel_spacing (float): Base voxel spacing
        radius (float): Particle radius
        
    Returns:
        tuple: (tomogram tensor, effective_voxel_spacing, scale_factor)
    """
    tomogram_wrapper = run.get_voxel_spacing(voxel_spacing).get_tomogram(tomo_type)
    z = zarr.open(store=tomogram_wrapper.zarr(), path="/", mode="r")
    
    if radius <= RESOLUTION_THRESHOLD:
        # Use highest resolution
        tomogram = z['0'][:]
        effective_voxel_spacing = voxel_spacing
        scale_factor = 1
    else:
        # Use medium resolution
        tomogram = z['1'][:]
        effective_voxel_spacing = voxel_spacing * 2  # Scale factor is 2 for level 1
        scale_factor = 2
        
    return torch.tensor(tomogram).to(DEVICE), effective_voxel_spacing, scale_factor

def process_all_runs(root, session_id, user_id, voxel_spacing):
    """Process all runs and save results to CSV."""
    results = []
    pick_id = 0
    
    for run in tqdm(root.runs):
        start_time = time.time()
        print(f"\nProcessing run: {run.meta.name}")
        
        # Process each particle type separately since they might need different resolutions
        for obj in root.pickable_objects:
            if not obj.is_particle:
                continue
                
            radius = obj.radius
            print(f"Processing {obj.name} with radius {radius}")
            
            # Get appropriate resolution data
            tomogram_tensor, effective_voxel_spacing, scale_factor = get_tomogram_data(
                run, voxel_spacing, radius)

            print(f"Using scale factor {scale_factor} (effective voxel spacing: {effective_voxel_spacing})")

            # Create segmentation at appropriate scale
            segmentation = create_hessian_particle_mask(tomogram_tensor, sigma=3)
            
            if torch.sum(segmentation) == 0:
                print(f"No particles detected in segmentation for {obj.name}")
                continue

            # Adjust radius for effective voxel spacing
            scaled_radius = radius / effective_voxel_spacing

            # Erode and dilate the segmentation
            dilated_mask = erode_dilate_mask(segmentation, scaled_radius)

            # Distance transform and local maxima detection
            distance = distance_transform(dilated_mask)
            local_max = local_maxima(distance, scaled_radius)

            # Convert tensors to numpy for watershed
            local_max_np = local_max.cpu().numpy()
            distance_np = distance.cpu().numpy()
            dilated_mask_np = dilated_mask.cpu().numpy()

            # Watershed segmentation
            markers, _ = ndi.label(local_max_np)
            watershed_labels = watershed(-distance_np, markers, mask=dilated_mask_np)

            # Extract region properties and scale coordinates back to original space
            centroids = []
            for region in regionprops(watershed_labels):
                # Scale the centroid coordinates back to original space
                centroid = np.array(region.centroid) * scale_factor
                centroids.append(centroid)  # ZYX order

            # Save centroids as picks and add to results
            if centroids:
                pick_set = run.get_picks(obj.name)
                if pick_set:
                    pick_set = pick_set[0]
                else:
                    pick_set = run.new_picks(obj.name, session_id, user_id)
                
                for centroid in centroids:
                    # Convert from ZYX to XYZ order and apply voxel spacing
                    x = centroid[2] * voxel_spacing  # Z -> X
                    y = centroid[1] * voxel_spacing  # Y -> Y
                    z = centroid[0] * voxel_spacing  # X -> Z
                    
                    # Add to results list
                    row = [pick_id, run.meta.name, obj.name, x, y, z]
                    results.append(row)
                    pick_id += 1
                    
                # Store pick set
                pick_set.points = [{'x': c[2] * voxel_spacing,
                                  'y': c[1] * voxel_spacing,
                                  'z': c[0] * voxel_spacing}
                                 for c in centroids]
                pick_set.store()
                print(f"Saved {len(centroids)} centroids for {obj.name}")
            else:
                print(f"No valid centroids found for {obj.name}")

        # Print timing for this run
        end_time = time.time()
        print(f"Run {run.meta.name} completed in {end_time - start_time:.2f} seconds")

    print(f"\nTotal picks found: {len(results)}")

    # Write results to CSV
    with open(OUTPUT_CSV_PATH, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["id", "experiment", "particle_type", "x", "y", "z"])
        writer.writerows(results)
    
    print(f"Results saved to {OUTPUT_CSV_PATH}")
    return results

In [6]:
# Run the processing
root = copick.from_file(copick_config_path)
results = process_all_runs(
    root=root,
    session_id="0",
    user_id="blobDetector",
    voxel_spacing=10
)

  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-5-4d784b12415b>:189: DeprecationWarning: get_tomogram is deprecated, use get_tomograms instead. Results may be incomplete
  tomogram_wrapper = run.get_voxel_spacing(voxel_spacing).get_tomogram(tomo_type)



Processing run: TS_5_4
Processing apo-ferritin with radius 60.0
Using scale factor 2 (effective voxel spacing: 20)
Mask shape for erosion: torch.Size([1, 1, 92, 315, 315])
Structuring element shape: torch.Size([1, 1, 7, 7, 7])
Padding size: 1


/usr/local/lib/python3.10/dist-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `CopickPoint` but got `dict` with value `{'x': 2997.4859234234236,...'z': 184.45382882882882}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5850.896236981688, ..., 'z': 895.194757364862}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4976.470588235294, ... 'z': 9.411764705882353}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4569.198329853863, ... 'z': 449.5240083507307}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4999.381443298969, ... 'z': 63.71134020618556}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 2382.9767441860467,... 'z': 67.81395348837209}` - serialized value may not be as expected
  Expec

Saved 190 centroids for apo-ferritin
Processing beta-amylase with radius 65.0
Using scale factor 2 (effective voxel spacing: 20)
Mask shape for erosion: torch.Size([1, 1, 92, 315, 315])
Structuring element shape: torch.Size([1, 1, 7, 7, 7])
Padding size: 1
Saved 190 centroids for beta-amylase
Processing beta-galactosidase with radius 90.0
Using scale factor 2 (effective voxel spacing: 20)
Mask shape for erosion: torch.Size([1, 1, 92, 315, 315])
Structuring element shape: torch.Size([1, 1, 9, 9, 9])
Padding size: 2


/usr/local/lib/python3.10/dist-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `CopickPoint` but got `dict` with value `{'x': 3003.210988102347, ...'z': 152.19624236299325}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 6075.890778871979, ...'z': 226.01611459265888}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 4472.970027247956, ... 'z': 79.98183469573115}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 2707.8159698754466,... 'z': 699.5139243299907}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 6063.361344537815, ... 'z': 55.33613445378151}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 3235.435409035409, ... 'z': 516.3203907203907}` - serialized value may not be as expected
  Expec

...

Saved 11 centroids for ribosome
Processing thyroglobulin with radius 130.0
Using scale factor 2 (effective voxel spacing: 20)
Mask shape for erosion: torch.Size([1, 1, 92, 315, 315])
Structuring element shape: torch.Size([1, 1, 13, 13, 13])
Padding size: 3


/usr/local/lib/python3.10/dist-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `CopickPoint` but got `dict` with value `{'x': 5435.050423349625, ... 'z': 798.2223618090452}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 3115.005736868944, ... 'z': 373.3853901070882}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5966.608695652174, ... 'z': 56.69565217391305}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5403.698781838317, ... 'z': 53.39977851605759}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 3364.640625, 'y': 8...359375, 'z': 178.734375}` - serialized value may not be as expected
  Expected `CopickPoint` but got `dict` with value `{'x': 5111.927831607083, ... 'z': 256.0841964584029}` - serialized value may not be as expected
  Expec

Saved 20 centroids for thyroglobulin
Processing virus-like-particle with radius 135.0
Using scale factor 2 (effective voxel spacing: 20)
Mask shape for erosion: torch.Size([1, 1, 92, 315, 315])
Structuring element shape: torch.Size([1, 1, 13, 13, 13])
Padding size: 3


100%|██████████| 3/3 [00:28<00:00,  9.40s/it]

Saved 20 centroids for virus-like-particle
Run TS_6_4 completed in 9.85 seconds

Total picks found: 1941
Results saved to submission.csv


In [9]:
results[:6]

[[0,
  'TS_5_4',
  'apo-ferritin',
  2997.4859234234236,
  360.1276276276276,
  184.45382882882882],
 [1,
  'TS_5_4',
  'apo-ferritin',
  5850.896236981688,
  4638.284069724617,
  895.194757364862],
 [2,
  'TS_5_4',
  'apo-ferritin',
  4976.470588235294,
  452.9411764705883,
  9.411764705882353],
 [3,
  'TS_5_4',
  'apo-ferritin',
  4569.198329853863,
  480.4801670146138,
  449.5240083507307],
 [4,
  'TS_5_4',
  'apo-ferritin',
  4999.381443298969,
  496.2886597938144,
  63.71134020618556],
 [5,
  'TS_5_4',
  'apo-ferritin',
  2382.9767441860467,
  719.2558139534884,
  67.81395348837209]]